In [76]:
import re

import pandas as pd
import torch
import torch.nn as nn
from random import randrange
from sklearn.preprocessing import LabelBinarizer
from _MatchNotFoundException import MatchNotFoundException
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from train_utils import AverageMeter

In [77]:
match_cols = ['season', 'round'] + \
['date', 'time', 'referee', 'home_team', 'away_team', 'hom_score', 'away_score'] + \
['home_team_coach'] + \
['home_player_' + str(i) for i in range(1, 12)] + \
['home_substitute_' + str(i) for i in range(1, 8)] + \
['away_team_coach'] + \
['away_player_' + str(i) for i in range(1, 12)] + \
['away_substitute_' + str(i) for i in range(1, 8)]

In [78]:
train = pd.read_csv('train.csv')

In [79]:
train

,season,round,year,month,day,hour,home_score,away_score,home_player_Sebastien Frey,home_player_Marco Amelia,...,away_coach_Roberto Mancini,away_coach_Rosario Pergolizzi,away_coach_Serse Cosmi,away_coach_Silvio Baldini,away_coach_Sinisa Mihajlovic,away_coach_Stefano Colantuono,away_coach_Stefano Pioli,away_coach_Walter Alfredo Novellino,away_coach_Walter Mazzarri,away_coach_Walter Zenga
0,0,1,2005,8,27,20,2,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,2005,8,27,18,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2005,8,28,15,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,3,22,2009,2,1,15,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1308,3,22,2009,2,1,15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1309,3,22,2009,2,1,15,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1310,3,22,2009,2,1,15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# utility methods
def get_column_names_containing_str(df: pd.DataFrame, substring: str) -> list[str]:
    return df.loc[:, df.columns.str.contains(substring)].columns.values.tolist()


def get_team_and_historical_index_from_match_team_id(match_team_id: str) -> (str, str):
    match_team_name = re.findall("\s+", match_team_id)[0]
    match_team_index = re.findall("\d+", match_team_id)[0]
    return match_team_name, match_team_index


def get_match_by_team_season_round(df: pd.DataFrame, team: str, season: int, round: int) -> pd.DataFrame:
    """Get the match played by the given team in the given season and round. If the team has not played any match in that round, an empty dataframe is returned."""
    return df[((df[f'home_team_{team}'] == 1) | (df[f'away_team_{team}'] == 1)) & (df['round'] == round) & (df['season'] == season)]


def get_last_n_matches_played_by_team_before_round_in_season(df: pd.DataFrame, team: str, season: int, round: int, n: int) -> pd.DataFrame:
    """Look in df for the last n matches played by the given team before the given round and season. A dataframe with exactly n element is returned.
    If n matches can't be found from the current season, the previous ones are iteratively considered, until n matches are found or the end
    of the dataframe is reached, in which case padding is applied to ensure a result size of n."""

    def exists_historical_matches_before_round_and_season(q_round: int, q_season: int) -> bool:
        if (q_season == 0) & (q_round <= 1):
            return False
        return True

    def decrement_round_in_season(c_round: int, c_season: int) -> (int, int):
        if c_round - 1 > 0:
            c_round -= 1
            return c_round, c_season
        c_season -= 1
        c_round = 38
        return c_round, c_season

    # print(f'fetching historical matches for team={team} before round={round} in season={season}')
    current_round, current_season = round, season
    result = pd.DataFrame()
    while True:
        if not exists_historical_matches_before_round_and_season(current_round, current_season):
            if result.empty:
                raise MatchNotFoundException
            return fill_with_padding(result)
        current_round, current_season = decrement_round_in_season(current_round, current_season)
        historical_match_at_current_round = get_match_by_team_season_round(df, team, current_season, current_round)
        if not historical_match_at_current_round.empty:
            result = pd.concat([result, historical_match_at_current_round])
            if len(result) == n:
                return result


def fill_with_padding(source: pd.DataFrame):
    if len(source) < 5:
        initial_len = len(source)
        padding = source.tail(1)
        for i in range(5 - len(source)):
            source = pd.concat([source, padding], ignore_index=True)
        # print(f'padding applied. Initial len: {initial_len} new_len: {len(source)}')
    return source


def get_playing_home_team_name(row: pd.DataFrame) -> str:
    team_columns = row.loc[:, [col for col in row.columns if col.startswith('home_team_')]]
    team_name = team_columns.where(team_columns == 1).dropna(axis=1).columns[0].replace('home_team_', '')
    return team_name


def get_playing_away_team_name(row: pd.DataFrame) -> str:
    team_columns = row.loc[:, [col for col in row.columns if col.startswith('away_team_')]]
    team_name = team_columns.where(team_columns == 1).dropna(axis=1).columns[0].replace('away_team_', '')
    return team_name

In [81]:
tot_num_of_feats = len(train.columns)
print(f'Total number of encoded features: {tot_num_of_feats}')

Total number of encoded features: 2590


In [82]:
# test historical retrieval
# idx = 751
# x = train.iloc[[idx]]
# team = get_playing_home_team_name(x)
# juventus = get_match_by_team_season_round(train, team, 1, 38)
# print(juventus)
# last_n_games_home = get_last_n_matches_played_by_team_before_round_in_season(train, team, x['season'].values[0], x['round'].values[0], 5)
# last_n_games_away = get_last_n_matches_played_by_team_before_round_in_season(train, team, x['season'].values[0], x['round'].values[0], 5)
# print(last_n_games_home.shape)
# print(last_n_games_away.shape)

In [83]:
del train
# del lb

### Dataset construction

We need to define a torch Dataset and torch Dataloader that will be used during training.

In [84]:
class SerieAFootballMatchesDataset(Dataset):
    history_len = 5

    def __init__(self, csv_file):
        self.dataframe = pd.read_csv(csv_file)

    def __len__(self) -> int:
        return len(self.dataframe)

    def __getitem__(self, idx):
        idx = self.scale_min_idx(idx)
        x = self.dataframe.iloc[[idx]]  # df
        y = self.dataframe[['result_home', 'result_draw', 'result_away']].iloc[0].values
        try: # if we are not able to fetch at least one historical match, then we switch to another index
            last_n_games_home, last_n_games_away = self.retrieve_historical_data(x)
            x, x_historical_home, x_historical_away, y = self.to_tensor(x, last_n_games_home, last_n_games_away, y)
            exp_num_of_features = len(self.dataframe.columns)
            if x.shape[0] != exp_num_of_features:
                self.showError(idx, x, x_historical_home, x_historical_away, y)
            if (x_historical_home.shape[0] != 5) | (x_historical_home.shape[1] != exp_num_of_features):
                self.showError(idx, x, x_historical_home, x_historical_away, y)
            if (x_historical_away.shape[0] != 5) | (x_historical_away.shape[1] != exp_num_of_features):
                self.showError(idx, x, x_historical_home, x_historical_away, y)
            if y.shape[0] != 3:
                self.showError(idx, x, x_historical_home, x_historical_away, y)
            return x, x_historical_home, x_historical_away, y
        except MatchNotFoundException:
            new_idx = randrange(0, len(self.dataframe))
            print(f'MatchNotFoundException for idx={idx}, switching to idx={new_idx}')
            return self.__getitem__(new_idx)

    def showError(self, index, x, x_historical_home, x_historical_away, y):
        print(f'error at index (scaled): {index} (unscaled): {self.unscale_min_idx(index)}')
        print(f'x: {x}')
        print(f'x.shape: {x.shape}')
        print(f'x_historical_home: {x_historical_home}')
        print(f'x_historical_home.shape: {x_historical_home.shape}')
        print(f'x_historical_away: {x_historical_away}')
        print(f'x_historical_away.shape: {x_historical_away.shape}')
        print(f'y: {y}')
        print(f'y.shape: {y.shape}')

    def scale_min_idx(self, idx: int) -> int:
        """Scale the given index to a range with a new minimum that allows for historical data retrieval"""
        old_min = 0
        old_max = len(self.dataframe)
        '''
        idx = 10 corresponds to the first match of the second round.
        This ensure the retrieval of at least 1 historical match.
        In the worst case scenario, padding will fill the other 4 historical slots.
        '''
        new_min = 10
        new_max = old_max
        return self.scale_idx(idx, old_min, old_max, new_min, new_max)

    def unscale_min_idx(self, idx: int) -> int:
        """Apply the inverse transformation of scale_min_idx"""
        old_min = 10
        old_max = len(self.dataframe)
        new_min = 0
        new_max = old_max
        return self.scale_idx(idx, old_min, old_max, new_min, new_max)

    def scale_idx(self, idx, old_min, old_max, new_min, new_max):
        """Scale the given index to a new range"""
        old_range = old_max - old_min
        new_range = new_max - new_min
        normalized_idx = (idx - old_min) / old_range
        return int(round(normalized_idx * new_range + new_min))

    def retrieve_historical_data(self, source: pd.DataFrame):
        """Retrieve historical data for home and away teams from source"""
        last_n_games_home = get_last_n_matches_played_by_team_before_round_in_season(
            self.dataframe, get_playing_home_team_name(source), source['season'].values[0], source['round'].values[0], self.history_len)
        last_n_games_away = get_last_n_matches_played_by_team_before_round_in_season(
            self.dataframe, get_playing_away_team_name(source), source['season'].values[0], source['round'].values[0], self.history_len)
        return last_n_games_home, last_n_games_away

    def to_tensor(self, x: pd.DataFrame, x_historical_home: pd.DataFrame, x_historical_away: pd.DataFrame,
               y: list[int]):
        x_tensor = torch.flatten(torch.tensor(x.values))
        x_historical_home_tensor = torch.tensor(x_historical_home.values)
        x_historical_away_tensor = torch.tensor(x_historical_away.values)
        y_tensor = torch.tensor(y)
        return x_tensor, x_historical_home_tensor, x_historical_away_tensor, y_tensor


## Training

In [85]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh = nn.Tanh()

    def forward(self, input, hidden):
        combined = torch.cat([input, hidden], dim=0)
        pre_hidden = self.linear(combined)
        hidden = self.tanh(pre_hidden)
        return hidden

    def init_hidden(self, minibatch_size):
        return torch.zeros(minibatch_size, self.hidden_size)


class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
            # nn.Softmax(dim=1) softmax is applied implicitly by CrossEntropyLoss
        )

    def forward(self, x):
        # 'x' is the combination of: 'x', 'x_historical_home', 'x_historical_away'
        # they all have size: minibatch_size x num_of_feats
        x = self.flatten(x) # just in case x was not flattened
        output = self.layers(x)
        return output


class HybridNetwork(nn.Module):
    def __init__(self, rnn_home_model: RNN, rnn_away_model: RNN, mlp_model: NeuralNetwork):
        super(HybridNetwork, self).__init__()
        self.rnn_home = rnn_home_model
        self.rnn_away = rnn_away_model
        self.mlp = mlp_model

    def forward(self, x, x_historical_home, x_historical_away):
        # 'x' comes in as:                minibatch_size x 1 x num_of_feats
        # 'x_historical_*' comes in as:   minibatch_size x 5 x num_of_feats
        # 'rnn_*_hidden' will be:         minibatch_size x num_of_feats
        batch_size = x.size(0)
        time_seq_len = x_historical_home.size(1)
        # RNN HOME FORWARD
        rnn_home_hidden = self.rnn_home.init_hidden(batch_size)
        # print(rnn_home_hidden)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_home_hidden[batch_idx] = self.rnn_home(
                    torch.flatten(x_historical_home[batch_idx, history_idx]),
                    rnn_home_hidden[batch_idx])
        # RNN AWAY FORWARD
        rnn_away_hidden = self.rnn_away.init_hidden(batch_size)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_away_hidden[batch_idx] = self.rnn_away(
                    torch.flatten(x_historical_away[batch_idx, history_idx]),
                    rnn_away_hidden[batch_idx])
        # MLP FORWARD
        # concat on the features dimension
        x_train = torch.cat([x, rnn_home_hidden, rnn_away_hidden], dim=1)
        y_hat = self.mlp(x_train)
        return y_hat

In [86]:
def train_epoch(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, loss_meter):
    for x, x_historical_home, x_historical_away, y in dataloader:
        optimizer.zero_grad()
        y_hat = model(x, x_historical_home, x_historical_away)
        loss = loss_fn(y.to(dtype=torch.float), y_hat)
        print(f'loss: {loss}')
        loss.backward()
        optimizer.step()
        loss_meter.update(val=loss.item(), n=x.shape[0])


def train_model(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, num_epochs: int):
    model.train()
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        train_epoch(model=model, dataloader=dataloader, optimizer=optimizer, loss_fn=loss_fn, loss_meter=loss_meter)
        print(f"Epoch {epoch + 1} completed. Training loss: {loss_meter.avg}")

In [87]:
train_dataset = SerieAFootballMatchesDataset(csv_file='train.csv')

In [88]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [89]:
learning_rate = 0.001
num_epochs = 3
hidden_size = 128
rnn_home = RNN(input_size=tot_num_of_feats, hidden_size=hidden_size)
rnn_away = RNN(input_size=tot_num_of_feats, hidden_size=hidden_size)
# we have two hidden states (for home and away team) plus all training features
nnInputSize = hidden_size * 2 + tot_num_of_feats
mlp = NeuralNetwork(nnInputSize)
model = HybridNetwork(rnn_home_model=rnn_home, rnn_away_model=rnn_away, mlp_model=mlp)
cross_entropy_loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
train_model(model=model, dataloader=train_dataloader, optimizer=optimizer, loss_fn=cross_entropy_loss_fn, num_epochs=num_epochs)

# Missing data
- We don't have data about new players that come to play in _Serie A_ during the course of the seasons. The model has to learn from zero context how important their contribution is for the outcome of the matches. If we were to considered multiple leagues, we could keep track of player transfers and maintain the history.
- We don't have data about cup matches played during the course of the seasons, like _Champions League_, _Europa League_ and _Coppa Italia_. Since they are very prestigious competitions and matches are usually very competitive, teams put a lot of effort in them and therefore can then perform worse in the championship.
- We don't have any type of player performance metric like who scored a goal, who was the assist man, red or yellow cards, goalkeeper's saves etc. so the model could face some difficulties in learning which player is important for the team.